# Building Spectral-based Model
In the following examples, we will build two typical spectral-based models (GCN and HGNN ) on graph and hypergraph, respectively.

GCN

In [ ]:
import dhg
import torch
import torch.nn as nn

class GCNConv(nn.Module):
    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        bias: bool = True,
        drop_rate: float = 0.5,
    ):
        super().__init__()
        self.act = nn.ReLU(inplace=True)
        self.drop = nn.Dropout(drop_rate)
        self.theta = nn.Linear(in_channels, out_channels, bias=bias)

    def forward(self, X: torch.Tensor, g: dhg.Graph) -> torch.Tensor:
        X = self.theta(X)
        X_ = g.smoothing_with_GCN(X)
        X_ = self.drop(self.act(X_))
        return X_

HGNN

In [ ]:
import dhg
import torch
import torch.nn as nn

class HGNNConv(nn.Module):
    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        bias: bool = True,
        drop_rate: float = 0.5,
    ):
        super().__init__()
        self.act = nn.ReLU(inplace=True)
        self.drop = nn.Dropout(drop_rate)
        self.theta = nn.Linear(in_channels, out_channels, bias=bias)

    def forward(self, X: torch.Tensor, hg: dhg.Hypergraph) -> torch.Tensor:
        X = self.theta(X)
        X_ = hg.smoothing_with_HGNN(X)
        X_ = self.drop(self.act(X_))
        return X_

# Building Spatial-based Model

Building GraphSAGE model

In [1]:
import dhg
import torch
import torch.nn as nn

class GraphSAGEConv(nn.Module):
    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        aggr: str = "mean",
        bias: bool = True,
        drop_rate: float = 0.5,
    ):
        super().__init__()
        assert aggr in ["mean"], "Currently, only mean aggregation is supported."
        self.aggr = aggr
        self.act = nn.ReLU(inplace=True)
        self.drop = nn.Dropout(drop_rate)
        if aggr == "mean":
            self.theta = nn.Linear(in_channels * 2, out_channels, bias=bias)
        else:
            raise NotImplementedError()

    def forward(self, X: torch.Tensor, g: dhg.Graph) -> torch.Tensor:
        if self.aggr == "mean":
            X_nbr = g.v2v(X, aggr="mean")
            X = torch.cat([X, X_nbr], dim=1)
        else:
            raise NotImplementedError()
        X_ = self.theta(X)
        X_ = self.drop(self.act(X_))
        return X_

c:\Users\biehl\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Building HGNN+ model

In [ ]:
import dhg
import torch
import torch.nn as nn

class HGNNPConv(nn.Module):
    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        bias: bool = True,
        drop_rate: float = 0.5,
    ):
        super().__init__()
        self.act = nn.ReLU(inplace=True)
        self.drop = nn.Dropout(drop_rate)
        self.theta = nn.Linear(in_channels, out_channels, bias=bias)

    def forward(self, X: torch.Tensor, hg: dhg.Hypergraph) -> torch.Tensor:
        X = self.theta(X)
        Y = hg.v2e(X, aggr="mean")
        X_ = hg.e2v(Y, aggr="mean")
        X_ = self.drop(self.act(X_))
        return X_

Building GAT model

In [ ]:
import dhg
import torch
import torch.nn as nn

class GATConv(nn.Module):
    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        bias: bool = True,
        drop_rate: float = 0.5,
        atten_neg_slope: float = 0.2,
    ):
        super().__init__()
        self.atten_dropout = nn.Dropout(drop_rate)
        self.atten_act = nn.LeakyReLU(atten_neg_slope)
        self.act = nn.ELU(inplace=True)
        self.theta = nn.Linear(in_channels, out_channels, bias=bias)
        self.atten_src = nn.Linear(out_channels, 1, bias=False)
        self.atten_dst = nn.Linear(out_channels, 1, bias=False)

    def forward(self, X: torch.Tensor, g: dhg.Graph) -> torch.Tensor:
        X = self.theta(X)
        x_for_src = self.atten_src(X)
        x_for_dst = self.atten_dst(X)
        e_atten_score = x_for_src[g.e_src] + x_for_dst[g.e_dst]
        e_atten_score = self.atten_dropout(self.atten_act(e_atten_score).squeeze())
        X_ = g.v2v(X, aggr="softmax_then_sum", e_weight=e_atten_score)
        X_ = self.act(X_)
        return X_

Building hypergraph convolution with different hyperedge weights model

In [ ]:
import dhg
import torch
import torch.nn as nn

class HGATConv(nn.Module):
    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        bias: bool = True,
        drop_rate: float = 0.5,
        atten_neg_slope: float = 0.2,
    ):
        super().__init__()
        self.atten_dropout = nn.Dropout(drop_rate)
        self.atten_act = nn.LeakyReLU(atten_neg_slope)
        self.act = nn.ELU(inplace=True)
        self.theta_vertex = nn.Linear(in_channels, out_channels, bias=bias)
        self.theta_hyperedge = nn.Linear(in_channels, out_channels, bias=bias)
        self.atten_vertex = nn.Linear(out_channels, 1, bias=False)
        self.atten_hyperedge = nn.Linear(out_channels, 1, bias=False)

    def forward(self, X: torch.Tensor, Y: torch.Tensor, hg: dhg.Hypergraph) -> torch.Tensor:
        X = self.theta_vertex(X)
        Y = self.theta_hyperedge(Y)
        x_for_vertex = self.atten_vertex(X)
        y_for_hyperedge = self.atten_hyperedge(Y)
        v2e_atten_score = x_for_vertex[hg.v2e_src] + y_for_hyperedge[hg.v2e_dst]
        e2v_atten_score = y_for_hyperedge[hg.e2v_src] + x_for_vertex[hg.e2v_dst]
        v2e_atten_score = self.atten_dropout(self.atten_act(v2e_atten_score).squeeze())
        e2v_atten_score = self.atten_dropout(self.atten_act(e2v_atten_score).squeeze())
        Y_ = hg.v2e(X, aggr="softmax_then_sum", v2e_weight=v2e_atten_score)
        X_ = hg.e2v(Y_, aggr="softmax_then_sum", e2v_weight=e2v_atten_score)
        X_ = self.act(X_)
        Y_ = self.act(Y_)
        return X_, Y_

Building Hybrid Operation Model

In [ ]:
import dhg
import torch
import torch.nn as nn

class HOMConv(nn.Module):
    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        bias: bool = True,
        drop_rate: float = 0.5,
    ):
        super().__init__()
        self.act = nn.ReLU(inplace=True)
        self.drop = nn.Dropout(drop_rate)
        self.theta = nn.Linear(in_channels, out_channels, bias=bias)

    def forward(self, X: torch.Tensor, g: dhg.Graph) -> torch.Tensor:
        X = self.theta(X)
        X_spectral = g.smoothing_with_GCN(X)
        X_spatial = g.v2v(X, aggr="mean")
        X_ = (X_spectral + X_spatial) / 2
        X_ = self.drop(self.act(X_))
        return X_

Building Hybrid Structure Model

In [ ]:
import dhg
import torch
import torch.nn as nn

class HSMConv(nn.Module):
    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        bias: bool = True,
        drop_rate: float = 0.5,
    ):
        super().__init__()
        self.act = nn.ReLU(inplace=True)
        self.drop = nn.Dropout(drop_rate)
        self.theta = nn.Linear(in_channels, out_channels, bias=bias)

    def forward(self, X: torch.Tensor, g: dhg.Graph, hg: dhg.Hypergraph) -> torch.Tensor:
        X = self.theta(X)
        X_g = g.v2v(X, aggr="mean")
        X_hg = hg.v2v(X, aggr="mean")
        X_ = (X_g + X_hg) / 2
        X_ = self.drop(self.act(X_))
        return X_